In [51]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [61]:
# Files
import input_net
import utils

# Packages
from os import path
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from transformers import AlbertConfig, AlbertForSequenceClassification, AlbertTokenizer
from transformers import BartConfig, BartForSequenceClassification, BartTokenizer
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
from transformers import CamembertConfig, CamembertForSequenceClassification, CamembertTokenizer
from transformers import DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer
from transformers import FlaubertConfig, FlaubertForSequenceClassification, FlaubertTokenizer
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer
from transformers import XLMConfig, XLMForSequenceClassification, XLMTokenizer
from transformers import XLMRobertaConfig, XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from transformers import XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer

# Optimizer -> busca info de com s'utilitza
from transformers import AdamW, get_linear_schedule_with_warmup

In [10]:
TRAIN = "data/train.csv"
TEST = "data/test.csv"
INPUT_NET = 'data/input.csv'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len = 70

if not path.exists(INPUT_NET):
    df = input_net.create_input(TRAIN, INPUT_NET, tokenizer, max_len))
else:
    df = pd.read_csv(INPUT_NET)

tokens_tensor, segments_tensor, attention_tensor, targets_tensor = utils.ToTensor(df)

In [45]:
MODEL_CLASSES = {
    'albert': (AlbertConfig, AlbertForSequenceClassification, AlbertTokenizer),
    'bart': (BartConfig, BartForSequenceClassification, BartTokenizer),
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'camembert': (CamembertConfig, CamembertForSequenceClassification, CamembertTokenizer),
    'distilbert': (DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer),
    'flaubert': (FlaubertConfig, FlaubertForSequenceClassification, FlaubertTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'xlm roberta': (XLMRobertaConfig, XLMRobertaForSequenceClassification, XLMRobertaTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

# Mirar aixo:: AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path)

In [ ]:
args = {
    'model_type': 'xlm',
    'model_name': 'bert-base-cased',
    'do_train': True,
    'do_eval': True,
    'max_seq_length': 70,
    'batch_size': 16, 

    'num_train_epochs': 4,
    'weight_decay': 0,
    'learning_rate': 4e-5,
    'adam_epsilon': 1e-8,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,
}

In [46]:
'''
tokenizer = tokenizer_class.from_pretrained(args['model_name'])

model = model_class.from_pretrained(args['model_name'])

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
'''

In [22]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [17]:
Predictions = model(input_ids=tokens_tensor[:5],token_type_ids=segments_tensor[:5],attention_mask=attention_tensor[:5])

In [24]:
F.softmax(model(input_ids=tokens_tensor[:5],token_type_ids=segments_tensor[:5],attention_mask=attention_tensor[:5])[0], dim=1)

tensor([[0.6671, 0.3329],
        [0.6624, 0.3376],
        [0.6661, 0.3339],
        [0.6886, 0.3114],
        [0.6588, 0.3412]], grad_fn=<SoftmaxBackward>)